# Imports

In [ ]:
import pandas                as pd
import numpy                 as np
import matplotlib.pyplot     as plt
import seaborn               as sns
from math                    import sqrt
from sklearn.linear_model    import LinearRegression
from sklearn.linear_model    import LassoCV
from sklearn.linear_model    import RidgeCV
from sklearn.metrics         import r2_score
from sklearn.metrics         import mean_squared_error
from sklearn.metrics         import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing   import StandardScaler
from IPython.core.display    import display, HTML
sns.set(style = "white", palette = "husl")
display(HTML("<style>.container { width:95% !important; }</style>"))
%matplotlib inline

# Table Of Contents

-----

1. [Reading In The Data](#Reading-In-The-Data)
    - [Overview](#Overview)

-----

2. [Feature Engineering](#Feature-Engineering)
    - [Transforming Numeric Data](#Transforming-Numeric-Data)
    - [Creating Segmental Features](#Creating-Segmental-Features)
    
-----

3. [Modeling](#Modeling)
    - [Functions](#Functions)
    - [Linear Regression](#Linear-Regression)
    - [Ridge](#Ridge)
    - [LASSO](#LASSO)

# Reading In The Data

In [ ]:
mri = pd.read_csv("../Data/mri_cleaned.csv")

## Overview

In [ ]:
mri.head()

The `smoker_status` column is still in the data, but we will not needing the column for the models because we turned it into a pair of dummy columns so we will drop the column here.

In [ ]:
# Checking the shape of the data

print(f"The shape of the dataset is: {mri.shape}")

In [ ]:
# Summary of column data types

mri.dtypes.value_counts()

# Feature Engineering

## Transforming Numeric Data

There are only four numeric columns in the data set: `age`, `lvesv`, `lvedv`, `lvef`.  Of the four, only `lvef` does not have any kind of a normal distribution: `age` is close to normally distributed, while `lvesv` and `lvedv` are log-normally distributed.

We cannot do anything to `lvedv` because that is my target variable, but we can take the log of `lvesv` (in this case the natural log).  We also tried squaring `age` but that did not affect the distribution in the way we hoped it would.

In [ ]:
# Take the natural log of `lvesv`.
# We chose to make it it's own column rather
# than overwrite the column.

mri["lvesv_log"] = mri["lvesv"].apply(lambda x: np.log(x))

In [ ]:
print(f"The shape of the dataset is: {mri.shape}")

[Top](#Table-Of-Contents)

## Creating Segmental Features

Because the model attempts to predict the end diastolic volume, we want it to be as accurate as it can be.  As part of that, we will try to use many combinations of features in an attempt to achieve high accuracy.

The data have 34 columns that we wish to engineer: a column measuring scarification and a column measuring ischemia.  Because there are so many of them, we felt the need to experiment with how they are passed into the model.  We are unable to create interaction columns, because there are zeros.  Instead, we elected to create segmental columns by summing similar columns together: we will compare the model's performance with the originals and with the segmental columns.

We used this image to guide our create of segmental columns:

<img src = "../Images/cardiac-segmentation-for-cardiac-perfusion-defects.jpg" alt = "Cardiac Segmentation" height = 750 width = 750>

Case courtesy of Dr Hamid Chalian, <a href="https://radiopaedia.org/">Radiopaedia.org</a>. From the case <a href="https://radiopaedia.org/cases/47102">rID: 47102</a>

In [ ]:
# Creating columns based off of the red, blue, green, and yellow
# I'm combining the apex and apical regions because there is only
# one region in the apex.

# New columns for scar tissue

mri["basal_he"]  = mri["ba_he"] + mri["bas_he"] + mri["bis_he"] \
                   + mri["bi_he"] + mri["bil_he"] + mri["bal_he"]
mri["mid_he"]    = mri["ma_he"] + mri["mas_he"] + mri["mis_he"] \
                   + mri["mi_he"] + mri["mil_he"] + mri["mal_he"]
mri["apical_he"] = mri["aa_he"] + mri["as_he"] + mri["ai_he"] \
                   + mri["al_he"] + mri["apex_he"]

# New columns for ischemia

mri["basal_ischemia"]  = mri["ba_ischemia"] + mri["bas_ischemia"] + mri["bis_ischemia"] \
                         + mri["bi_ischemia"] + mri["bil_ischemia"] + mri["bal_ischemia"]
mri["mid_ischemia"]    = mri["ma_ischemia"] + mri["mas_ischemia"] + mri["mis_ischemia"] \
                         + mri["mi_ischemia"] + mri["mil_ischemia"] + mri["mal_ischemia"]
mri["apical_ischemia"] = mri["aa_ischemia"] + mri["as_ischemia"] + mri["ai_ischemia"] \
                         + mri["al_ischemia"]

print(f"The shape of the dataset is: {mri.shape}")

[Top](#Table-Of-Contents)

It will be easier when modeling to have two data sets: one with the original features and one with the features I engineered.


In [ ]:
# Defining the original dataframe

mri_org = mri.drop(labels = ["lvesv_log", "basal_he", "mid_he",
                             "apical_he", "basal_ischemia",
                             "mid_ischemia", "apical_ischemia"],
                   axis = 1)

# Defining the dataframe with only new features

mri_eng = mri.drop(labels = ['lvesv', 'ba_he', 'bas_he', 'bis_he','bi_he', 
                             'bil_he', 'bal_he', 'ma_he', 'mas_he', 'mis_he', 
                             'mi_he', 'mil_he','mal_he', 'aa_he', 'as_he', 
                             'ai_he', 'al_he', 'apex_he', 'ba_ischemia',
                             'bas_ischemia', 'bis_ischemia', 'bi_ischemia', 
                             'bil_ischemia','bal_ischemia', 'ma_ischemia', 
                             'mas_ischemia', 'mis_ischemia','mi_ischemia', 
                             'mil_ischemia', 'mal_ischemia', 'aa_ischemia', 
                             'as_ischemia', 'ai_ischemia', 'al_ischemia'],
                   axis = 1)

# Checking to make sure the two have different numbers of columns

print(f"The shape of the dataframe with original features is: {mri_org.shape}")
print(f"The shape of the dataframe with new features is     : {mri_eng.shape}")

[Top](#Table-Of-Contents)

# Modeling

Before we can start modeling, we have to perform a train-test split.  A train-test split allows us to train our data on one subset of the dataframe and train on another subset.

Since I have two versions of the dataframe, I will have to train-test split on both.

In [ ]:
# Setting up my X and y variables for the original

X  = mri_org.drop("lvedv", axis = 1)
y  = mri_org["lvedv"]

# Setting up my X and y variables for the new

X_eng = mri_eng.drop("lvedv", axis = 1)
y_eng = mri_eng["lvedv"]

In [ ]:
# Train-test splitting mri_og

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y,
                                                    random_state = 42,
                                                    test_size    = 0.25) 

# Train-test splitting mri_new

X_train_eng, X_test_eng, y_train_eng, y_test_eng = train_test_split(X_eng, 
                                                                    y_eng,
                                                                    random_state = 42,
                                                                    test_size    = 0.25) 

## Functions

In [ ]:
def r2_adj(X, y_true, y_predicted):
    r2          = r2_score(y_true, y_predicted)
    numerator   = (1 - r2) * (len(y) - 1)
    denominator = (len(y) - len(X.columns)) - 1
    quotient    = numerator / denominator
    r2_adj      = 1 - quotient
    return r2_adj

In [ ]:
# This function will be my primary model evaluation
# method for this project. 

def model_evaluation(X, y_true, y_predicted):
    rmse = sqrt(mean_squared_error(y_true, y_predicted))
    mae  = mean_absolute_error(y_true, y_predicted)
    r2   = r2_score(y_true, y_predicted)
    print(f"The root mean squared error is : {rmse}")
    print(f"The mean absolute error is     : {mae}")
    print(f"The r2 score is                : {r2}")
    print(f"The adjusted r2 score is       : {r2_adj(X, y_true, y_predicted)}")

In [ ]:
def model_evaluation_nor2adj(y_true, y_predicted):
    rmse = sqrt(mean_squared_error(y_true, y_predicted))
    mae  = mean_absolute_error(y_true, y_predicted)
    r2   = r2_score(y_true, y_predicted)
    print(f"The root mean squared error is : {rmse}")
    print(f"The mean absolute error is     : {mae}")
    print(f"The r2 score is                : {r2}")

## Linear Regression

In [ ]:
# Instantiating the linear regression

lin_reg = LinearRegression()

In [ ]:
# Fitting the linear regression to the original
# (non-engineered) subset

lin_reg.fit(X_train, y_train);

In [ ]:
# Generating my predictions

y_pred  = lin_reg.predict(X_test)

In [ ]:
# Evaluating my original set

model_evaluation(X_test, y_test, y_pred)

In [ ]:
pd.DataFrame(data  = lin_reg.coef_,
             index = X_test.columns)

In [ ]:
# Fitting the linear regression to the subset
# with the features I engineered

lin_reg.fit(X_train_eng, y_train_eng);

In [ ]:
# Generating my predictions

y_pred_eng  = lin_reg.predict(X_test_eng)

In [ ]:
# Evaluating my new set

model_evaluation(X_test_eng, y_test_eng, y_pred_eng)

In [ ]:
pd.DataFrame(data  = lin_reg.coef_,
             index = X_test_eng.columns)

## Ridge

Ridge regression works by imposing a penalty on the coefficients.  For that reason, the data has to be scaled because small differences in the data can make the penalties enormous.  We used `StandardScaler` to scale the data.

In [ ]:
# Instantiating the standard scaler

ss = StandardScaler()

In [ ]:
# Transforming the original set

# Fit-transforming my X_train features

X_train_ss     = ss.fit_transform(X_train)

# Transforming my X_test variables

X_test_ss  = ss.transform(X_test)

In [ ]:
# Transforming the engineered set

# Fit-transforming my X_train_eng features

X_train_eng_ss = ss.fit_transform(X_train_eng)

# Transforming my X_test_eng features

X_test_eng_ss = ss.transform(X_test_eng)

In [ ]:
# Instantiating the ridge model

ridge = RidgeCV()

In [ ]:
# Fitting the ridge model to the training data

ridge.fit(X_train_ss, y_train);

In [ ]:
# Generating Predictions

y_pred = ridge.predict(X_test_ss)

In [ ]:
model_evaluation_nor2adj(y_test, y_pred)

In [ ]:
# Fitting the ridge model to the data
# with engineered features

ridge.fit(X_train_eng_ss, y_train_eng);

In [ ]:
y_pred_eng = ridge.predict(X_test_eng_ss)

In [ ]:
model_evaluation_nor2adj(y_test_eng, y_pred_eng)

## LASSO

LASSO is similar to ridge in that it also has to be scaled to work properly.  We will be using `StandardScaler` again.

In [ ]:
# Transforming the original set

# Fit-transforming my X_train features

X_train_ss     = ss.fit_transform(X_train)

# Transforming my X_test variables

X_test_ss  = ss.transform(X_test)

In [ ]:
# Transforming the engineered set

# Fit-transforming my X_train_eng features

X_train_eng_ss = ss.fit_transform(X_train_eng)

# Transforming my X_test_eng features

X_test_eng_ss = ss.transform(X_test_eng)

In [ ]:
# Instantiating the LASSO model

lasso = LassoCV()

In [ ]:
# Fitting the lasso to my training data
# without engineered features

lasso.fit(X_train_ss, y_train)

In [ ]:
y_pred = lasso.predict(X_test_ss)

In [ ]:
model_evaluation_nor2adj(y_test, y_pred)

In [ ]:
# Fitting the lasso to my training data
# with engineered features

lasso.fit(X_train_eng_ss, y_train_eng)

In [ ]:
y_pred_eng = lasso.predict(X_test_eng_ss)

In [ ]:
model_evaluation_nor2adj(y_test_eng, y_pred_eng)